In [49]:
import os
import time
import math
import itertools
from tqdm import tqdm
import yaml
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds
from hypernet_MLP import Hypernet_MLP
from hypernet_trans import Hypernet_trans
from functions_hv_python3 import HyperVolume

In [8]:
def set_seed(seed=702):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(702)

In [9]:
case = "_Ex_7_1"

# Const

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [11]:
GROUND_TRUTH_FILE = f"../4_Pareto_front/test/{case}/pf_dynamic_true.npy" 

if os.path.exists(GROUND_TRUTH_FILE):
    pf_true = np.load(GROUND_TRUTH_FILE)
    print(f"✅ Đã tải ground truth Pareto front từ: {GROUND_TRUTH_FILE}, Shape: {pf_true.shape}")
else:
    print(f"❌ KHÔNG tìm thấy file ground truth tại: {GROUND_TRUTH_FILE}")

✅ Đã tải ground truth Pareto front từ: ../4_Pareto_front/test/_Ex_7_1/pf_dynamic_true.npy, Shape: (20, 2)


# Def

In [12]:
import autograd.numpy as np
from scipy.optimize import minimize            
        
class Projection:
    def __init__(self, cons, bounds, dim, proj_type='euclid'):
        self.cons = cons
        self.bounds = bounds
        self.dim = dim
        self.proj_type = proj_type
        
        if self.proj_type == 'qplus':
            self.objective_func = self._obj_positive_diff
        elif self.proj_type == 'euclid':
            self.objective_func = self._obj_l2_norm
        else:
            print(f"Phép chiếu {self.objective_func} không cài đặt, chọn 'qplus' hoặc 'euclid'")

    def _obj_l2_norm(self, x, y):
        return np.sqrt(np.sum((x - y)**2))
    
    def _obj_positive_diff(self, x, y):
        v = np.maximum(y - x, 0) 
        return np.sum(v**2)

    def project(self, target_point):
        init_point = np.random.rand(1, self.dim).tolist()[0]
        
        res = minimize(
            self.objective_func,
            init_point,
            args=(target_point, ),
            constraints=self.cons,
            bounds=self.bounds,
            options={'disp': False}
        )
        
        optim_point = res.x
        
        if self.proj_type == 'qplus':
            return target_point - np.maximum(target_point - optim_point, 0)
        else:
            return optim_point

class Problem():
    def __init__(self, f, dim_x, dim_y, proj_C, proj_Qplus):
        self.f = f
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.proj_C = proj_C
        self.proj_Qplus = proj_Qplus
    
    def objective_func(self, x):
        vals = [func(x) for func in self.f]
        return np.concatenate(vals)  

In [13]:
def f1(x):    return x[0]
def f2(x):    return (x[0] - 1)**2
#--------------- C --------------------#
def c1(x):    return x[0]
def c2(x):    return -x[0] + 1
#--------------- Q --------------------#
def q1(y):    return 0.2**2 - (y[0] - 0.4)**2 - (y[1] - 0.4)**2

def q_plus(y):
    center = .4
    radius_sq = 0.2**2  
    dx = np.maximum(0, y[0] - center)
    dy = np.maximum(0, y[1] - center)
    return radius_sq - (dx**2 + dy**2)
# Hàm dùng cho Projection 
cons_C = ({'type': 'ineq', 'fun' : c1,},
          {'type': 'ineq', 'fun' : c2,})
dim_x = 1
cons_Q = ({'type': 'ineq', 'fun' : q1,},)
cons_Qplus = ({'type': 'ineq', 'fun': q_plus},)
dim_y = 2

# Setup Projections
proj_C_handler = Projection(cons=cons_C, bounds=None, dim=dim_x, proj_type='euclid')
proj_Q_handler = Projection(cons=cons_Q, bounds=None, dim=dim_y, proj_type='qplus')
# Setup Problem
prob = Problem(
    f=[f1, f2], 
    dim_x=dim_x, dim_y=dim_y,
    proj_C=proj_C_handler.project,
    proj_Qplus=proj_Q_handler.project
)

In [16]:
def evaluate_objectives_single(functions, x_tensor):
    vals = []
    for func in functions:
        val = func(x_tensor)
        if not torch.is_tensor(val):
            val = torch.tensor(val, dtype=torch.float32)
        vals.append(val)
    return torch.stack(vals).reshape(-1)

def train_hypernet(hypernet, prob, z_star, 
                   num_epochs=1000, 
                   lr=1e-3, 
                   num_partitions=100, 
                   lr_step_size=300, 
                   lr_gamma=0.5,
                   # --- THAM SỐ THUẬT TOÁN 2-A (Monotonic Penalty) ---
                   beta_C_0=1.0,    # Giá trị khởi tạo cho C
                   beta_C_max=1000.0, # Giá trị tối đa cho C
                   rho_C=1.01,      # Tỷ lệ tăng cho C 
                   
                   beta_Q_0=1.0,    # Giá trị khởi tạo cho Q 
                   beta_Q_max=1000.0, # Giá trị tối đa cho Q
                   rho_Q=1.01,      # Tỷ lệ tăng cho Q
                   verbose=True): 
    
    # 1. Khởi tạo
    optimizer = optim.Adam(hypernet.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_step_size, gamma=lr_gamma)
    
    # Đảm bảo z_star chuẩn shape
    z_star_tensor = torch.tensor(z_star, dtype=torch.float32).view(1, -1)
    
    # Khởi tạo hệ số phạt
    beta_C = beta_C_0
    beta_Q = beta_Q_0
    
    angle_step = (math.pi / 2) / num_partitions
    
    if verbose:
        print(f"=== TRAIN HYPERNET (Algorithm 2-A: Monotonic Penalty) ===")
        print(f"Constraint C: Start {beta_C_0} -> Max {beta_C_max} (Rate {rho_C})")
        print(f"Constraint Q: Start {beta_Q_0} -> Max {beta_Q_max} (Rate {rho_Q})")
    
    for epoch in range(num_epochs):
        hypernet.train()
        optimizer.zero_grad()
        
        # --------------------------------------------------------
        # 1. Lấy mẫu phân tầng (Stratified Sampling)
        # --------------------------------------------------------
        starts = torch.arange(num_partitions) * angle_step
        noise = torch.rand(num_partitions) * angle_step
        thetas = starts + noise 
        
        r_batch_np = np.stack([np.cos(thetas.numpy()), np.sin(thetas.numpy())], axis=1)
        r_tensor_batch = torch.tensor(r_batch_np, dtype=torch.float32)
        
        # --------------------------------------------------------
        # 2. Lan truyền xuôi & Tính Loss 
        # --------------------------------------------------------
        
        # Forward pass 
        x_pred_list = []
        for i in range(num_partitions):
            r_single = r_tensor_batch[i].unsqueeze(0)
            x_single = hypernet(r_single)
            x_pred_list.append(x_single)
            
        x_vec_batch = torch.cat(x_pred_list, dim=0) # Batch output
        
        # Loop tính Loss thành phần 
        x_np_batch = x_vec_batch.detach().cpu().numpy()
        loss_C_list = []
        loss_Q_list = []
        y_pred_list = []
        
        for i in range(num_partitions):
            x_i_tensor = x_vec_batch[i].reshape(-1) 
            x_i_np = x_i_tensor.detach().cpu().numpy()
            
            # a. Loss C: ||x - P_C(x)||^2
            x_proj_i_np = prob.proj_C(x_i_np)
            x_proj_i_tensor = torch.tensor(x_proj_i_np, dtype=torch.float32)
            loss_C_list.append(torch.sum((x_i_tensor - x_proj_i_tensor)**2))
            
            # b. F(x)
            y_pred_i = evaluate_objectives_single(prob.f, x_i_tensor)
            y_pred_list.append(y_pred_i)
            
            # c. Loss Q: ||y - P_Q+(y)||^2
            y_i_np = y_pred_i.detach().cpu().numpy()
            y_proj_i_np = prob.proj_Qplus(y_i_np)
            y_proj_i_tensor = torch.tensor(y_proj_i_np, dtype=torch.float32)
            loss_Q_list.append(torch.sum((y_pred_i - y_proj_i_tensor)**2))
        
        # Tính trung bình Loss (Mean L_C, Mean L_Q)
        y_pred_batch = torch.stack(y_pred_list)
        L_bar_C = torch.mean(torch.stack(loss_C_list))
        L_bar_Q = torch.mean(torch.stack(loss_Q_list))
        
        # --------------------------------------------------------
        # 3. Tính Loss Mục tiêu Chebyshev 
        # --------------------------------------------------------
        diff = y_pred_batch - z_star_tensor
        weighted_diff = r_tensor_batch * diff
        max_vals, _ = torch.max(weighted_diff, dim=1)
        L_Obj = torch.mean(max_vals)
        
        # --------------------------------------------------------
        # 4. Tổng hợp và Cập nhật 
        # --------------------------------------------------------
        # L_Total = L_Obj + β_C * L_C + β_Q * L_Q
        total_loss = L_Obj + (beta_C * L_bar_C) + (beta_Q * L_bar_Q)
        
        total_loss.backward()
        optimizer.step()
        scheduler.step()
        
        # --------------------------------------------------------
        # 5. Cập nhật hệ số Phạt (Step 30-31 of Alg 2-A)
        # --------------------------------------------------------
        # Cả hai hệ số đều TĂNG dần
        beta_C = min(beta_C_max, beta_C * rho_C)
        beta_Q = min(beta_Q_max, beta_Q * rho_Q)
        
        # Logging
        if verbose and epoch % 100 == 0:
            current_lr = scheduler.get_last_lr()[0]
            print(f"Epoch {epoch}: Total={total_loss.item():.3f} "
                  f"(Obj={L_Obj.item():.4f}, C={L_bar_C.item():.5f}, Q={L_bar_Q.item():.5f}) "
                  f"|| BetaC={beta_C:.1f}, BetaQ={beta_Q:.1f}")
            
    return hypernet

In [25]:
def calculate_med(pf_pred, pf_true):
    """Tính Mean Error Distance (Euclidean)"""
    pf_pred = np.array(pf_pred)
    pf_true = np.array(pf_true)
    if pf_pred.shape != pf_true.shape:
        return np.inf
    distances = np.linalg.norm(pf_pred - pf_true, axis=1)
    return np.mean(distances)
def calculate_hv_score(pareto_f, prob, ref_point):
    """Tính Hypervolume, loại bỏ điểm vi phạm ràng buộc Q+"""
    valid_points = []
    tol = 1e-3
    for point in pareto_f:
        # Kiểm tra feasibility với Q+
        point_proj = prob.proj_Qplus(point)
        dist_Q = np.linalg.norm(point - point_proj)
        
        # Kiểm tra nằm trong vùng Reference
        is_dominated_by_ref = np.all(point < ref_point)
        
        if dist_Q < tol and is_dominated_by_ref:
            valid_points.append(point.tolist())
            
    if len(valid_points) < 2: return 0.0
    
    hv = HyperVolume(ref_point)
    return hv.compute(valid_points)

def evaluate_model_all(hypernet, prob, test_rays, pf_true, ref_point):
    """
    Đánh giá model một lần duy nhất cho cả MED và HV.
    """
    hypernet.eval()
    pf_pred = []
    rays_tensor = torch.tensor(test_rays, dtype=torch.float32, device=device)
    
    with torch.no_grad():
        for i in range(len(rays_tensor)):
            r_single = rays_tensor[i].unsqueeze(0)
            x_raw = hypernet(r_single).squeeze().cpu().numpy()
            
            # Chiếu lên C để đảm bảo x hợp lệ
            x_proj = prob.proj_C(x_raw)
            
            # Tính f(x)
            val = [func(x_proj) for func in prob.f]
            pf_pred.append(val)
            
    pf_pred = np.array(pf_pred)
    
    # Tính các chỉ số
    med_score = calculate_med(pf_pred, pf_true)
    hv_score = calculate_hv_score(pf_pred, prob, ref_point)
    
    return med_score, hv_score, pf_pred

# Config

In [46]:
param_grid = {
    'lr': [1e-3, 5e-4],
    'num_epochs': [500, 1000, 2000],
    
    'num_partitions': [20],
    
    # Tham số thuật toán 2-A: Tăng dần penalty
    'beta_C_0': [10],
    'beta_Q_0': [5],
    'rho_C': [1.03], 
    'rho_Q': [1.01, 1.03],
    
    # Cố định Max để tránh grid quá lớn 
    'beta_C_max': [1000.0],
    'beta_Q_max': [100.0, 500.0, 1000.0]
}

z_star = np.array([0.1496, 0.0797])
ref_point = np.array([1.3912, 1.4064])
config_path='../4_Pareto_front/config.yaml'
with open(config_path, 'r') as f:
    cfg = yaml.safe_load(f)
test_rays = np.array(cfg['data']['test_ray'])
pf_true = np.load(f"../4_Pareto_front/test/{case}/pf_dynamic_true.npy")

# Run

In [42]:
models = ["trans", "MLP"]
INDICATOR = "MED"

In [43]:
results = []
best_scores_tracker = {}
save_dir = f"model/{case}"

In [52]:
results = []

for model_name in models:
    print(f"\n🔹 Đang train Model: {model_name} | Ưu tiên: {INDICATOR}")
    
    # Khởi tạo giá trị tốt nhất tùy theo Indicator
    if INDICATOR == "MED":
        current_best_score = float('inf')  # MED tìm Min
    else:
        current_best_score = -1.0          # HV tìm Max
        
    current_best_config = None
    
    # Tạo lưới tham số
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    for idx, params in tqdm(enumerate(param_combinations)):
        print(f"\n   >>> Config {idx+1}/{len(param_combinations)}: {params}")
        
        # 1. Init Model
        if model_name == "MLP":
            model = Hypernet_MLP(ray_hidden_dim=32, out_dim=dim_x, n_tasks=2).to(device)
        else:
            model = Hypernet_trans(ray_hidden_dim=32, out_dim=dim_x, n_tasks=2).to(device)
        
        # 2. Train (Sử dụng hàm train_hypernet đã có của bạn)
        start_time = time.time()
        trained_model = train_hypernet(
            model, prob, z_star, 
            num_epochs=params['num_epochs'],
            lr=params['lr'],
            num_partitions=params['num_partitions'], 
            beta_C_0=params['beta_C_0'],
            beta_C_max=params['beta_C_max'],
            rho_C=params['rho_C'],
            beta_Q_0=params['beta_Q_0'],
            beta_Q_max=params['beta_Q_max'],
            rho_Q=params['rho_Q'],
            verbose=True 
        )
        train_time = time.time() - start_time

        # 3. Đánh giá đa mục tiêu
        med, hv, pf_pred = evaluate_model_all(trained_model, prob, test_rays, pf_true, ref_point)
        
        # Chọn score để so sánh dựa trên INDICATOR
        score_to_compare = med if INDICATOR == "MED" else hv
        
        print(f"      ⏱️ Time: {train_time:.1f}s | 📏 MED: {med:.6f} | 📈 HV: {hv:.6f}")

        # 4. Lưu log
        res = {
            'model_type': model_name,
            'config_id': idx,
            'params': params,
            'med': med,
            'hv': hv,
            'time': train_time
        }
        results.append(res)

        # 5. Cập nhật Best Model
        is_best = False
        if INDICATOR == "MED":
            if score_to_compare < current_best_score:
                current_best_score = score_to_compare
                is_best = True
        else: # INDICATOR == "HV"
            if score_to_compare > current_best_score:
                current_best_score = score_to_compare
                is_best = True
        
        if is_best:
            current_best_config = params
            save_path = f"{save_dir}/best_{model_name}_{INDICATOR}.pth"
            torch.save(trained_model.state_dict(), save_path)
            print(f"      🏆 New Best {INDICATOR} Found! Saved to: {save_path}")

    print(f"\n✅ Hoàn thành {model_name}. Best {INDICATOR}: {current_best_score:.6f}")


🔹 Đang train Model: trans | Ưu tiên: MED


0it [00:00, ?it/s]


   >>> Config 1/36: {'lr': 0.001, 'num_epochs': 500, 'num_partitions': 20, 'beta_C_0': 10, 'beta_Q_0': 5, 'rho_C': 1.03, 'rho_Q': 1.01, 'beta_C_max': 1000.0, 'beta_Q_max': 100.0}
=== TRAIN HYPERNET (Algorithm 2-A: Monotonic Penalty) ===
Constraint C: Start 10 -> Max 1000.0 (Rate 1.03)
Constraint Q: Start 5 -> Max 100.0 (Rate 1.01)
Epoch 0: Total=7.808 (Obj=1.0219, C=0.08906, Q=1.17918) || BetaC=10.3, BetaQ=5.0
Epoch 100: Total=0.175 (Obj=0.1754, C=0.00000, Q=0.00000) || BetaC=198.0, BetaQ=13.7
Epoch 200: Total=0.139 (Obj=0.1378, C=0.00000, Q=0.00003) || BetaC=1000.0, BetaQ=36.9
Epoch 300: Total=0.138 (Obj=0.1370, C=0.00000, Q=0.00002) || BetaC=1000.0, BetaQ=99.9
Epoch 400: Total=0.138 (Obj=0.1376, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=100.0


1it [00:33, 33.97s/it]

      ⏱️ Time: 33.9s | 📏 MED: 0.005940 | 📈 HV: 1.367230
      🏆 New Best MED Found! Saved to: model/_Ex_7_1/best_trans_MED.pth

   >>> Config 2/36: {'lr': 0.001, 'num_epochs': 500, 'num_partitions': 20, 'beta_C_0': 10, 'beta_Q_0': 5, 'rho_C': 1.03, 'rho_Q': 1.01, 'beta_C_max': 1000.0, 'beta_Q_max': 500.0}
=== TRAIN HYPERNET (Algorithm 2-A: Monotonic Penalty) ===
Constraint C: Start 10 -> Max 1000.0 (Rate 1.03)
Constraint Q: Start 5 -> Max 500.0 (Rate 1.01)
Epoch 0: Total=1.892 (Obj=0.6707, C=0.00494, Q=0.23444) || BetaC=10.3, BetaQ=5.0
Epoch 100: Total=0.141 (Obj=0.1409, C=0.00000, Q=0.00002) || BetaC=198.0, BetaQ=13.7
Epoch 200: Total=0.139 (Obj=0.1379, C=0.00000, Q=0.00002) || BetaC=1000.0, BetaQ=36.9
Epoch 300: Total=0.140 (Obj=0.1390, C=0.00000, Q=0.00001) || BetaC=1000.0, BetaQ=99.9
Epoch 400: Total=0.144 (Obj=0.1406, C=0.00000, Q=0.00001) || BetaC=1000.0, BetaQ=270.3


...

      ⏱️ Time: 129.0s | 📏 MED: 0.003967 | 📈 HV: 1.364969

   >>> Config 35/36: {'lr': 0.0005, 'num_epochs': 2000, 'num_partitions': 20, 'beta_C_0': 10, 'beta_Q_0': 5, 'rho_C': 1.03, 'rho_Q': 1.03, 'beta_C_max': 1000.0, 'beta_Q_max': 500.0}
=== TRAIN HYPERNET (Algorithm 2-A: Monotonic Penalty) ===
Constraint C: Start 10 -> Max 1000.0 (Rate 1.03)
Constraint Q: Start 5 -> Max 500.0 (Rate 1.03)
Epoch 0: Total=0.564 (Obj=0.4326, C=0.00000, Q=0.02628) || BetaC=10.3, BetaQ=5.2
Epoch 100: Total=0.167 (Obj=0.1666, C=0.00000, Q=0.00000) || BetaC=198.0, BetaQ=99.0
Epoch 200: Total=0.142 (Obj=0.1414, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=500.0
Epoch 300: Total=0.141 (Obj=0.1405, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=500.0
Epoch 400: Total=0.139 (Obj=0.1386, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=500.0
Epoch 500: Total=0.139 (Obj=0.1391, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=500.0
Epoch 600: Total=0.139 (Obj=0.1385, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=500.0
Epoch

35it [41:18, 116.92s/it]

      ⏱️ Time: 128.0s | 📏 MED: 0.004279 | 📈 HV: 1.370035

   >>> Config 36/36: {'lr': 0.0005, 'num_epochs': 2000, 'num_partitions': 20, 'beta_C_0': 10, 'beta_Q_0': 5, 'rho_C': 1.03, 'rho_Q': 1.03, 'beta_C_max': 1000.0, 'beta_Q_max': 1000.0}
=== TRAIN HYPERNET (Algorithm 2-A: Monotonic Penalty) ===
Constraint C: Start 10 -> Max 1000.0 (Rate 1.03)
Constraint Q: Start 5 -> Max 1000.0 (Rate 1.03)
Epoch 0: Total=0.573 (Obj=0.4307, C=0.00000, Q=0.02839) || BetaC=10.3, BetaQ=5.2
Epoch 100: Total=0.174 (Obj=0.1742, C=0.00000, Q=0.00000) || BetaC=198.0, BetaQ=99.0
Epoch 200: Total=0.140 (Obj=0.1396, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=1000.0
Epoch 300: Total=0.139 (Obj=0.1386, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=1000.0
Epoch 400: Total=0.139 (Obj=0.1381, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=1000.0
Epoch 500: Total=0.139 (Obj=0.1390, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=1000.0
Epoch 600: Total=0.138 (Obj=0.1379, C=0.00000, Q=0.00000) || BetaC=1000.0, BetaQ=1000.

36it [43:26, 72.40s/it] 

      ⏱️ Time: 127.6s | 📏 MED: 0.004527 | 📈 HV: 1.367440

✅ Hoàn thành MLP. Best MED: 0.003160


In [59]:
df = pd.json_normalize(results, sep='_')
df.columns = df.columns.str.replace('params_', '')
df.to_csv(f"exp/{case}.csv", index=False)

In [62]:
df.sort_values(by="med").head(10)

,model_type,config_id,med,hv,time,lr,num_epochs,num_partitions,beta_C_0,beta_Q_0,rho_C,rho_Q,beta_C_max,beta_Q_max
48,MLP,12,0.003160,1.360008,119.099762,0.0010,2000,20,10,5,1.03,1.01,1000.0,100.0
42,MLP,6,0.003338,1.367491,58.565843,0.0010,1000,20,10,5,1.03,1.01,1000.0,100.0
51,MLP,15,0.003456,1.364696,127.740639,0.0010,2000,20,10,5,1.03,1.03,1000.0,100.0
36,MLP,0,0.003457,1.365596,28.211488,0.0010,500,20,10,5,1.03,1.01,1000.0,100.0
66,MLP,30,0.003586,1.369481,127.844858,0.0005,2000,20,10,5,1.03,1.01,1000.0,100.0
45,MLP,9,0.003591,1.363416,58.903976,0.0010,1000,20,10,5,1.03,1.03,1000.0,100.0
12,trans,12,0.003618,1.364491,138.113011,0.0010,2000,20,10,5,1.03,1.01,1000.0,100.0
30,trans,30,0.003670,1.362916,137.681154,0.0005,2000,20,10,5,1.03,1.01,1000.0,100.0
33,trans,33,0.003810,1.357969,139.238878,0.0005,2000,20,10,5,1.03,1.03,1000.0,100.0
69,MLP,33,0.003967,1.364969,129.021076,0.0005,2000,20,10,5,1.03,1.03,1000.0,100.0


# Viz

In [ ]:
import pandas as pd
import ast 
import seaborn as sns

In [ ]:
if isinstance(df_results['params'].iloc[0], str):
    df_results['params'] = df_results['params'].apply(ast.literal_eval)

# Tách cột params thành các cột riêng biệt (lr, num_epochs, ...)
params_df = pd.json_normalize(df_results['params'])

# Ghép lại với cột score để vẽ
df_plot = pd.concat([df_results[['score']], params_df], axis=1)

param_cols = params_df.columns

# Thiết lập kích thước khung hình
plt.figure(figsize=(20, 5))

# Vẽ vòng lặp
for i, col in enumerate(param_cols):
    plt.subplot(1, len(param_cols), i + 1)
    
    # Vẽ scatter plot (biểu đồ chấm)
    sns.scatterplot(data=df_plot, x=col, y='score', s=100, color='blue')
    
    # Vẽ đường xu hướng (tùy chọn, giúp dễ nhìn trend)
    # sns.regplot(data=df_plot, x=col, y='score', scatter=False, color='red')
    
    plt.title(f'Score vs {col}')
    plt.xlabel(col)
    plt.ylabel('Score (MSE)')
    plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()